In [ ]:
# |default_exp daft_embeddings

In [ ]:
# | hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| export

import daft
import timm
import torch
import numpy as np
from PIL import Image
from pathlib import Path
from einops import rearrange
from dataclasses import dataclass, field
from typing import Callable
from tqdm import tqdm
import gc
from datasets import load_dataset

import shutil
import memray

In [ ]:
%load_ext memray

In [ ]:
#| export

daft.set_execution_config(enable_native_executor=True,
                          default_morsel_size=1
                          )

MAX_IMAGES = 2000

In [ ]:
#| hide

tmp_path = Path("./tmp-test-images")
shutil.rmtree(tmp_path, ignore_errors=True)
tmp_path.mkdir(parents=True, exist_ok=True)

dataset_name = "kvriza8/microscopy_images"

dataset = load_dataset(dataset_name, split="train", streaming=True)

for i, example in enumerate(tqdm(iter(dataset))):  # Use iterator to avoid full load
    if i >= MAX_IMAGES:
        break
    image = example["image"]
    image.save(tmp_path / f"{i}.png")

2000it [00:38, 51.71it/s] 


In [ ]:
#| export

@dataclass
class TimmEmbedder:
    """
    embed an image with any timm model that supports this

    Reference: https://huggingface.co/docs/timm/main/en/feature_extraction#pooled
    """
    model_name: str
    device: torch.device = field(init=False)
    dtype: torch.dtype = field(init=False)
    _model: Callable = field(init=False)
    _transforms: Callable = field(init=False)
    _amp_autocast: Callable = field(default=None, init=False, repr=False)
    _instance: "TimmEmbedder" = field(default=None, init=False, repr=False)

    def __new__(cls, model_name):
        if not hasattr(cls, "_instance") or cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance


    def __post_init__(self):
        if hasattr(self, "_model"):  # Avoid re-initialization
            return
        # initialize model and transforms
        self._model = timm.create_model(self.model_name, pretrained=True, num_classes=0)
        cfg = self._model.pretrained_cfg
        self._transform = timm.data.create_transform(**timm.data.resolve_data_config(cfg))

        # set device, dtype, and autocast function
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        self.dtype = torch.bfloat16 if self.device.type == "cuda" and torch.cuda.is_bf16_supported() else (
            torch.float16 if self.device.type == "cuda" else torch.float32
        )
        print(f"Inference device: {self.device} with dtype: {self.dtype}")

        # optimize model for inference
        self._model = self._model.eval().to(device=self.device, dtype=self.dtype)
        # self._model = torch.jit.optimize_for_inference(torch.jit.script(self._model))
        self._model = torch.compile(self._model)


    def __call__(self, image: torch.Tensor | Image.Image) -> np.array:
            """transform image, run inference, extract embedding as 1D array"""
            image = self._transform(image).to(self.device, self.dtype).unsqueeze(0)
            with torch.no_grad():
                emb = self._model(image)
            result = emb.detach().cpu().float().numpy().squeeze()
            del image, emb
            torch.cuda.empty_cache()
            gc.collect()
            return result


In [ ]:
#| hide

# it's a one-stop shop
# e = TimmEmbedder("mobilenetv3_large_100")
# e(torch.rand((3,256,256)))

In [ ]:
#| export

@daft.udf(return_dtype=daft.DataType.list(daft.DataType.float32()),
          memory_bytes=int(6e9)) # 6 Gb resource request
class EmbedImages:
    """run timm embedder on an image column"""
    def __init__(self, model_name: str):
        self.model_name = model_name
        self.embedder = TimmEmbedder(self.model_name)

    def __call__(self, images_col):
        images = [rearrange(im, "h w c -> c h w") for im in images_col.to_pylist()]
        ## Note: expect images are different sizes
        ## could maybe speed up by doing the resize transform separately,
        ## then doing batch inference
        ## Example: https://colab.research.google.com/github/Eventual-Inc/Daft/blob/main/tutorials/mnist.ipynb
        return [self.embedder(
            torch.tensor(im, dtype=self.embedder.dtype)) for im in images]

## Test it out

In [ ]:
#| hide

glob = tmp_path.as_posix() +"/*.png"
images_df = daft.from_glob_path(glob).with_column_renamed("path", "path_full_img")

images_df = images_df.with_column("image", daft.col("path_full_img"
                                 ).url.download().image.decode(
                                     mode="RGB", on_error="null"))

/home/willsa/git/bedmap-dev/.venv/lib/python3.12/site-packages/daft/context.py:168: UserWarning: Daft is configured to use the new NativeRunner by default as of v0.4.0. If you are encountering any regressions, please switch back to the legacy PyRunner via `daft.context.set_runner_py()` or by setting the env variable `DAFT_RUNNER=py`. We appreciate you filing issues and helping make the NativeRunner better: https://github.com/Eventual-Inc/Daft/issues
  warnings.warn(


## inference time for some embedding models on CPU on my laptop

model_name | 50 images | 500 images | 1000 images | 2000 images | 2kimg-1cpu-compile | 2kimg-1cpu-optimize_for_inference
---------- | --------- | ---------- | ----------- | ----------- | --------- | ----
mobilenetv3_large_100 | 0m02s |  0m8s | 0m12s | 0m25s | 8m05s | OOM
vit_small_patch14_reg4_dinov2.lvd142m | 0m26s | 4m26s | 8m52s | 17m17s | 29m03s | -
vit_base_patch14_reg4_dinov2.lvd142m | 1m20s | 11m48s | 23m29s | OOM | OOM | -
vit_large_patch14_reg4_dinov2.lvd142m | 4m04s | 38m30s | OOM | OOM | - | -

In [ ]:
#| hide

MODEL_NAME = "mobilenetv3_large_100"
EmbedImagesWithModel = EmbedImages.with_init_args(MODEL_NAME)

images_df = images_df.with_column("embed", EmbedImagesWithModel(daft.col("image")))

In [ ]:
# %%memray_flamegraph --native --max-memory-records 20

images_df = images_df.collect()

Inference device: cpu with dtype: torch.float32


🗡️ 🐟 Project: 00:00 

🗡️ 🐟 Project: 00:00 

KeyboardInterrupt: 

In [ ]:
#| hide

shutil.rmtree(tmp_path)